In [1]:
import pandas as pd
import numpy as np
import random
# import shap
import matplotlib.pyplot as plt
%matplotlib inline
import sys
sys.path.append('../')
from modules import utils, constants
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize, LabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report 
from sklearn.metrics import roc_curve, auc

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




Invalid MIT-MAGIC-COOKIE-1 key/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [2]:
# SEED = constants.SEED
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [3]:
#train_df = pd.read_csv('../data/train_set_basic.csv')
train_df = pd.read_csv('../new_data/train_set_missingness_0.3.csv')
# val_df = pd.read_csv('../data/noisiness/0.1/validation_set.csv')
# # train_df = pd.concat([train_df_orig, val_df]).reset_index(drop=True)
train_df = train_df.fillna(-1)
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train.head()
#train_df.label.value_counts()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody
0,1,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,...,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0
2,0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0
3,0,0.0,-1.0,-1.0,-1.0,0.0,-1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0
4,1,0.0,0.0,0.0,-1.0,1.0,-1.0,-1.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0


In [4]:
test_df = pd.read_csv('../new_data/test_set_constant.csv')
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]
X_test.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody
0,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,1,1,4,1,0,0,0,1,0,0


#### Decision Tree

In [5]:
# from sklearn.tree import DecisionTreeClassifier, export_graphviz
# dt = DecisionTreeClassifier(random_state=SEED).fit(X_train, y_train)
# y_pred_dt = dt.predict(X_test)
# test_df_dt = pd.DataFrame()
# test_df_dt['y_actual'] = y_test
# test_df_dt['y_pred'] = y_pred_dt
# test_df_dt.isna().sum()

In [6]:
# acc_dt2, f1_dt2, roc_auc_dt2 = utils.test(test_df_dt['y_actual'], test_df_dt['y_pred'])
# acc_dt2, f1_dt2, roc_auc_dt2

In [7]:
# utils.compute_feature_importance(dt, X_train)

#### SHAP

In [8]:
# explainer = shap.TreeExplainer(dt)
# shap_values = explainer.shap_values(X_test)
# len(shap_values)

In [9]:
# shap.summary_plot(shap_values, X_test, max_display=30)

#### Random Forest

In [10]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(random_state=SEED).fit(X_train, y_train)
# y_pred_rf = rf.predict(X_test)
# test_df_rf = pd.DataFrame()
# test_df_rf['y_actual'] = y_test
# test_df_rf['y_pred'] = y_pred_rf
# test_df_rf.isna().sum()

In [11]:
# acc_rf2, f1_rf2, roc_auc_rf2 = utils.test(test_df_rf['y_actual'], test_df_rf['y_pred'])
# acc_rf2, f1_rf2, roc_auc_rf2

#### XGBoost

In [12]:
import xgboost as xgb
xg = xgb.XGBClassifier(random_state=constants.SEED).fit(X_train, y_train)
y_pred_xg = xg.predict(X_test)
test_df_xg = pd.DataFrame()
test_df_xg['y_actual'] = y_test
test_df_xg['y_pred'] = y_pred_xg
test_df_xg.isna().sum()

y_actual    0
y_pred      0
dtype: int64

In [13]:
acc_xg2, f1_xg2, roc_auc_xg2 = utils.test(test_df_xg['y_actual'], test_df_xg['y_pred'])
acc_xg2, f1_xg2, roc_auc_xg2

(98.78571428571429, 98.78568890909067, 98.78667659567077)